# Create a sphere asset with variants for the display color and geom

In [1]:
from pxr import Usd, UsdGeom

In [2]:
import os
usd_scene_dir = '..\usd_scenes'
usd_sphere_filename='asset.usda'
usd_sphere_filepath = os.path.join(usd_scene_dir, usd_sphere_filename)

# Create a usda layer containing just the geometry of a ball

In [6]:
# Execute only if you have already created a stage and need to redo it.
try:
    del stage
except NameError:
    pass

In [7]:
# Create stage
stage = Usd.Stage.CreateNew(usd_sphere_filepath)

# Create a transform with a sphere primitive inside
asset = stage.DefinePrim('/asset', 'Xform')

# Set transform as default primitive in case this usda is referenced
stage.SetDefaultPrim(asset)

# Save and show result
stage.GetRootLayer().Save()
print stage.GetRootLayer().ExportToString()

#usda 1.0
(
    defaultPrim = "asset"
)

def Xform "asset"
{
}




## Add variantset and variants

In [8]:
variantset_name_list = asset.GetVariantSets().GetNames()
for variantset_name in variantset_name_list:
    print variantset_name
    variant_set = sphere.GetVariantSet(variantset_name)
    print variant_set

In [9]:
# Create a variant set only if it does not exists
variantset_name_list = asset.GetVariantSets().GetNames()
variantset_name = 'shadingVariant'
if variantset_name not in variantset_name_list:
    variant_set = asset.GetVariantSets().AddVariantSet('shadingVariant')

# Create each variant only if it does not exists
variant_list = {
    'red':{
        'color':[(1,0,0)],
        'geom':'Cube'
    }, 
    'blue':{
        'color':[(0,0,1)],
        'geom':'Sphere'
    }, 
    'green':{
        'color':[(0,1,0)],
        'geom':'Cone'
    }
}


for variant_name, variant_option in variant_list.iteritems():
    # Add variant if needed
    if variant_name not in variant_set.GetVariantNames():
        variant_set.AddVariant(variant_name)
        
    # Modify color attribute in the context of the variant
    variant_set.SetVariantSelection(variant_name)
    with variant_set.GetVariantEditContext():
        geom = stage.DefinePrim('/asset/geom', variant_option['geom'])
        
        #sphereSchema = UsdGeom.Sphere(geom)
        #colorAttr = sphereSchema.GetDisplayColorAttr()
        
        colorAttr = UsdGeom.Gprim.Get(stage, '/asset/geom').GetDisplayColorAttr()
        colorAttr.Set(variant_option['color'])

# Save and show result
stage.GetRootLayer().Save()
print stage.GetRootLayer().ExportToString()

#usda 1.0
(
    defaultPrim = "asset"
)

def Xform "asset" (
    variants = {
        string shadingVariant = "red"
    }
    prepend variantSets = "shadingVariant"
)
{
    variantSet "shadingVariant" = {
        "blue" {
            def Sphere "geom"
            {
                color3f[] primvars:displayColor = [(0, 0, 1)]
            }

        }
        "green" {
            def Cone "geom"
            {
                color3f[] primvars:displayColor = [(0, 1, 0)]
            }

        }
        "red" {
            def Cube "geom"
            {
                color3f[] primvars:displayColor = [(1, 0, 0)]
            }

        }
    }
}




In [10]:
!usdview {usd_sphere_filepath}